In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from time import perf_counter


In [2]:
NUM_NODES=1
conf = SparkConf().setMaster("spark://10.99.250.19:7077").setAppName("Adult")
conf.set("spark.executor.memory", "20G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "8")
conf.set("spark.cores.max", f"{NUM_NODES*8}")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# conf.set("spark.default.parallelism", "16")
# create a Spark Session instead of a Spark Context
spark_context =  SparkContext.getOrCreate(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/05 14:54:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load and parse the data file into an RDD of LabeledPoint.
rdd_list = []
for i in range(NUM_NODES):
    rdd_list.append(MLUtils.loadLibSVMFile(spark_context, '/work/li.baol/data/a9a'))
if NUM_NODES == 1:
    train_data = rdd_list[0]
else:
    train_data = spark_context.union(rdd_list)
# test_data = MLUtils.loadLibSVMFile(spark_context, '/work/li.baol/data/a9a.t')
# Split the data into training and test sets (30% held out for testing)
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

In [4]:
train_data, rdd_list

(PythonRDD[4] at RDD at PythonRDD.scala:53,
 [PythonRDD[4] at RDD at PythonRDD.scala:53])

In [5]:
train_data = train_data.map(lambda x: x if x.label == 1 else LabeledPoint(0, x.features))
# test_data = test_data.map(lambda x: x if x.label == 1 else LabeledPoint(0, x.features))

In [6]:
num_feature = train_data.take(1)[0].features.size
num_feature

123

In [7]:
# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
categorical = {}
for i in range(num_feature):
    categorical[i] = 2 # two types
    
start = perf_counter()

model = DecisionTree.trainClassifier(train_data, numClasses=2, categoricalFeaturesInfo=categorical,
                                    impurity='gini', maxDepth=8)
end = perf_counter()
end-start                                    

5.454712767153978

In [ ]:
# Evaluate model on test instances and compute test error
predictions = model.predict(train_data.map(lambda x: x.features))
predictions

In [ ]:
predictions.take(15), train_data.map(lambda x: x.label).take(15)

In [ ]:
labelsAndPredictions = train_data.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] == lp[1]).count() / float(train_data.count())
print('Test Error = ' + str(testErr))

In [ ]:
# predsAndLabels = predictions.zip(test_data.map(lambda lp: lp.label))
# # testMSE = predsAndLabels.map(lambda lp: (lp[1] - lp[0]) * (lp[1] - lp[0])).sum() /\
# #     float(test_data.count())
# metrics = BinaryClassificationMetrics(predsAndLabels)    
# print(f'AUC = {metrics.areaUnderROC}')


In [ ]:
predsAndLabels.take(10)